In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

from keras.preprocessing import image
from keras.layers import Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras.layers import Dropout

In [ ]:
'''
Inspired by paper, Joint Face Detection and Alignment using
Multi-task Cascaded Convolutional Networks
'''

In [ ]:
def build_Onet(input_tensor_shape):
    model = Sequential()
    model.add(Conv2D(32,(3,3),activation = 'relu'), input_shape= input_tensor_shape, name = 'conv_1')
    model.add(MaxPooling2D(pool_size=(3, 3)), name = 'max_pooling2d_1')
    
    model.add(Conv2D(64, (3, 3), activation='relu'), name = 'conv_2')
    model.add(MaxPooling2D(pool_size=(3, 3)), name = 'max_pooling2d_2')
    
    model.add(Conv2D(64, (3, 3), activation='relu'), name = 'conv_3')
    model.add(MaxPooling2D(pool_size=(2, 2)), name = 'max_pooling2d_2')
    
    model.add(Conv2D(128, (3, 3), activation='relu')name = 'conv_4')
    
    model.add(Flatten(), name = 'flatten')
    
    model.add(Dense(256, activation='relu',name='fc1_Dense'))
    model.add(Dropout(0.5, name='dropout_1'))
    
    model.add(Dense(128, activation='relu',name='fc2_Dense'))
    model.add(Dropout(0.5, name='dropout_2'))
    
    model.add(Dense(1, activation='sigmoid',name='output_layer'))
    return model

In [ ]:
def save_model_to_json(model,model_save_path):
    model_json = model.to_json()
    with open(model_save_path, 'w') as json_file:
        json_file.write(model_json)

In [ ]:
def training_data_shuffle(x_train, y_train):
    random_index = np.random.permutation(len(y_train))
    x_shuffle = []
    y_shuffle = []
    for i in range(len(y_train)):
        x_shuffle.append(x_train[random_index[i]])
        y_shuffle.append(y_train[random_index[i]])
    x = np.array(x_shuffle)
    y = np.array(y_shuffle)
    
    return x, y 

In [ ]:
# build model
img_rows, img_cols, img_channel = 48, 48, 3
input_tensor_shape=(img_rows, img_cols, img_channel)
model = build_Onet(input_tensor_shape)

# print model
for i, layer in enumerate(model.layers):
    print(i, layer.name)

# save model
save_model_to_json(model,'./onet_model.json')

In [ ]:
# load data
x_train = np.load('./x_train.npy')
y_train = np.load('./y_train.npy') 
x_test = np.load('./x_test.npy')
y_test = np.load('./y_test.npy')
# data shuffle
(x_train, y_train) = training_data_shuffle(x_train, y_train)
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)


In [ ]:
# compile the model
model.compile(optimizer=SGD(lr=0.00001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

# set train Generator
datagen = ImageDataGenerator(rotation_range=30,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True)
datagen.fit(x_train)

In [ ]:
# trainning process
nb_epoch = 3
batch_size = 32
checkpointer = ModelCheckpoint(filepath= './ResNet50_weights.hdf5', verbose=1, monitor='val_acc',save_best_only=False, save_weights_only=True)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch = x_train.shape[0],
                    epochs=nb_epoch,
                    validation_data = (x_test, y_test),
                    callbacks=[checkpointer])